In [5]:
import pandas as pd
import plotly.express as px 
from urllib.request import urlopen
import json
import janitor
import plotly.io as pio
import pandas_geojson as pdg
from json import loads, dumps
pio.renderers.default='notebook'

In [6]:
asthmaca = pd.read_csv('Sheets\\asthma_redownload_utf8.csv')
asthmaca = asthmaca.clean_names()
asthmaca = asthmaca.query('strata == \'Total population\'')
asthmaca = asthmaca.query('strata_name == \'All ages\'')
asthmaca = asthmaca.query('county != \'California\'')
asthmaca = asthmaca.query('year != 2015')
asthmaca = asthmaca.query('year != 2020')
asthmaca

,county,year,strata,strata_name,age_group,number_of_hospitalizations,age_adjusted_hospitalization_rate,comment
709,Alameda,2016,Total population,All ages,All ages,904,5.9,NaN
710,Alpine,2016,Total population,All ages,All ages,0,0.0,NaN
711,Amador,2016,Total population,All ages,All ages,13,3.4,NaN
712,Butte,2016,Total population,All ages,All ages,78,4.0,NaN
713,Calaveras,2016,Total population,All ages,All ages,10,NaN,Rate not available due to statistical instability
...,...,...,...,...,...,...,...,...
2886,Tulare,2019,Total population,All ages,All ages,210,4.2,NaN
2887,Tuolumne,2019,Total population,All ages,All ages,21,5.4,NaN
2888,Ventura,2019,Total population,All ages,All ages,252,3.0,NaN
2889,Yolo,2019,Total population,All ages,All ages,57,2.8,NaN


In [13]:
with open('Sheets\\California_County_Boundaries.json') as jsonfile:
    geojson = json.load(jsonfile)

In [8]:
geodf = pd.json_normalize(geojson, record_path='features')
geodf.head()

,type,properties.OBJECTID,properties.CountyName,properties.AdminRegion,properties.FireMAR,properties.LawMAR,properties.State_FIPS_ID,properties.County_FIPS_ID,properties.Shape__Area,properties.Shape__Length,geometry.type,geometry.coordinates
0,Feature,1,Alameda,Coastal,2,2,06,001,3.082165e+09,432059.180712,MultiPolygon,"[[[[-122.268526827671, 37.9044620205069], [-12..."
1,Feature,2,Alpine,Inland,4,4,06,003,3.145871e+09,274621.121060,Polygon,"[[[-119.934325016754, 38.8088244189646], [-119..."
2,Feature,3,Amador,Inland,4,4,06,005,2.559998e+09,357482.565247,Polygon,"[[[-120.257663940095, 38.5803311275207], [-120..."
3,Feature,4,Butte,Inland,3,3,06,007,7.338660e+09,526729.272631,Polygon,"[[[-121.63426641472, 40.0012746362627], [-121...."
4,Feature,5,Calaveras,Inland,4,4,06,009,4.352160e+09,371781.055548,Polygon,"[[[-120.209810650774, 38.5003340621002], [-120..."


In [9]:
fipmap = geodf.set_index('properties.CountyName')['properties.County_FIPS_ID'].to_dict()
asthmaca['fip'] = asthmaca['county'].map(fipmap)
asthmaca['fip'] = asthmaca['fip'].astype(str)
asthmaca

,county,year,strata,strata_name,age_group,number_of_hospitalizations,age_adjusted_hospitalization_rate,comment,fip
709,Alameda,2016,Total population,All ages,All ages,904,5.9,NaN,001
710,Alpine,2016,Total population,All ages,All ages,0,0.0,NaN,003
711,Amador,2016,Total population,All ages,All ages,13,3.4,NaN,005
712,Butte,2016,Total population,All ages,All ages,78,4.0,NaN,007
713,Calaveras,2016,Total population,All ages,All ages,10,NaN,Rate not available due to statistical instability,009
...,...,...,...,...,...,...,...,...,...
2886,Tulare,2019,Total population,All ages,All ages,210,4.2,NaN,107
2887,Tuolumne,2019,Total population,All ages,All ages,21,5.4,NaN,109
2888,Ventura,2019,Total population,All ages,All ages,252,3.0,NaN,111
2889,Yolo,2019,Total population,All ages,All ages,57,2.8,NaN,113


In [ ]:
fig = px.choropleth(
    asthmaca,
    geojson=geojson,
    featureidkey='properties.County_FIPS_ID',
    locations='fip',
    color='age_adjusted_hospitalization_rate',
    animation_frame='year',
    range_color=(0, 10),
    hover_data=['county'],
    width=1200,
    labels={
        'age_adjusted_hospitalization_rate':'Age Adjusted Hospitalization Rate'
    },
    color_continuous_scale='YlGn',
)

fig.update_geos(fitbounds="locations", visible=False)

fig.show()

In [12]:
fig.write_html('StrataTotalHospGeo.html')